In [177]:
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_2 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-05670e26-425c-4505-8824-ef9ea437d9c0',
    'IBM_API_KEY_ID': 'MaNADuPAUY6W0HC-8FxqIqpgji1yUUm6WVPlm9ulZIOW',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': 'nystesting-donotdelete-pr-yczufeevob9sat',
    'FILE': 'ManhattanLatLong.csv'
}

bingMapsKey = "AvDnxBjHKE2Y9SU1HRUpB6-79B0Bk_ldW-LCvqWkLWedsqWrlBX0kvkvl4JurmDO"


In [178]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='994f2a2f-d34c-428b-8963-154fd9fac775', project_access_token='p-a6c7ad6d54b86c48f27bb6924d28f925b70b5c08')
pc = project.project_context

## FourSquare

In [179]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

In [180]:
%%capture
!conda install -c conda-forge folium

In [181]:
# @hidden_cell
# FourSquare API credentials
CLIENT_ID = 'UYWTHYG53L3ENHJ4O50HDEHZKE51X0NPSR3YCXC1I0JCDGRA' # your Foursquare ID
CLIENT_SECRET = 'OPBF0AHVEAL11QB0HZUIWHP2BAUWQLZY4U0ZF10USQQVF0PJ' # your Foursquare Secret

In [182]:
# import necessary packages to work with spatial data in Python
import os
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.colors import ListedColormap
import json
from tabulate import tabulate
#import earthpy as et 

In [183]:
###### Retrieve all categories and sub-categories from FourSquare
VERSION = '20180604'
url = 'https://api.foursquare.com/v2/venues/categories?client_id={}&client_secret={}&v={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION)
results = requests.get(url).json()

# assign relevant part of JSON to categories
categories = results['response']['categories']

# tranform categories into a dataframe
dfCategories = json_normalize(categories)

# Each category in the DF contains sub-categories. We want all the
# categories and sub-categories flattened into one DF.
# Create an empty list. We'll keep appending categories and 
# sub-categories to that list, then we'll turn that list into a DF
allCategoriesList = []

nRowCount = 0
for row in dfCategories.itertuples():
    # First, put the parent category into the list
    # Use the category name for both the parent and category names
    rowData = [row.name, row.name, row.id]
    parentCategoryName = row.name;
    allCategoriesList.append(rowData)

    # Make a dictionary of the sub-categories of this category
    subCats = row.categories
    nRowCount = nRowCount + 1

    # Iterate over the sub-categories, adding each one to the list
    # Include the parent category's name
    for subCatRow in subCats:
        subCatID = subCatRow['id']
        subCatName = subCatRow['name']
        rowData = [subCatName, parentCategoryName, subCatID]
        allCategoriesList.append(rowData)

# Now make a DF of the flattened categories and sub-categories
dfAllCategories = pd.DataFrame(allCategoriesList)
dfAllCategories.columns = ['name', 'parentCategoryName', 'id']

# Set the index of the DF to the category/sub-category name, because we'll
# retrieve IDs by name
dfAllCategories = dfAllCategories.set_index(['name'])

print("There are " + str(dfAllCategories.shape[0]) + " categories and sub-categories in FourSquare")
# Show the data
print(tabulate(dfAllCategories, headers=['Name', 'Parent', 'ID']))


There are 466 categories and sub-categories in FourSquare
Name                                      Parent                       ID
----------------------------------------  ---------------------------  ------------------------
Arts & Entertainment                      Arts & Entertainment         4d4b7104d754a06370d81259
Amphitheater                              Arts & Entertainment         56aa371be4b08b9a8d5734db
Aquarium                                  Arts & Entertainment         4fceea171983d5d06c3e9823
Arcade                                    Arts & Entertainment         4bf58dd8d48988d1e1931735
Art Gallery                               Arts & Entertainment         4bf58dd8d48988d1e2931735
Bowling Alley                             Arts & Entertainment         4bf58dd8d48988d1e4931735
Casino                                    Arts & Entertainment         4bf58dd8d48988d17c941735
Circus                                    Arts & Entertainment         52e81612bcbc57f1066b79e7
Come

### Read in CSV files

### A user of this notebook can define which categories they want to analyze and map. For each category, they should also define what color they want map markers to be for that category, and which icon to use.

In [378]:
# You can change and/or add categories to explore in categoriesToFind
categoriesToFind = ['Movie Theater', 'Juice Bar']
for aCategory in categoriesToFind:
    print('{} category\'s ID is {}'.format(aCategory, dfAllCategories.loc[aCategory]['id']))
    
# For each category listed above, define the map marker color and icon to use
# Icons are available from these sites:
#   https://fontawesome.com/icons?d=gallery&m=free
#   https://getbootstrap.com/docs/3.3/components/
# The fields are category, color, icon_prefix, and icon_name
categoryMapSpecs = [{'category':categoriesToFind[0], 'color':'red', 'icon_prefix':'fa', 'icon_name':'fa-film'},
                    {'category':categoriesToFind[1], 'color':'orange', 'icon_prefix':'glyphicon', 'icon_name':'glyphicon-cutlery'}
                   ]
dfMapMarkers = pd.DataFrame.from_dict(categoryMapSpecs)
dfMapMarkers.set_index('category', inplace=True)
#print(dfMapMarkers.loc[categoriesToFind[0]].index)
print('Color is {}'.format(dfMapMarkers.loc[categoriesToFind[0]].color))

Movie Theater category's ID is 4bf58dd8d48988d17f941735
Juice Bar category's ID is 4bf58dd8d48988d112941735
Color is red


In [185]:
import io

# Read the file containing the overlapping rectangles for Manhattan
wrapper = io.TextIOWrapper(project.get_file("ManhattanLatLong.csv"), encoding='utf-8')
dfManhattanLatLon = pd.read_csv(wrapper)
dfManhattanLatLon.head()

,SWLAT,SWLON,NELAT,NELON
0,40.702142,-74.020323,40.707819,-74.002770
1,40.708046,-74.020688,40.715301,-73.981388
2,40.715203,-74.020655,40.724255,-73.978491
3,40.724190,-74.014841,40.730639,-73.977660
4,40.730671,-74.014009,40.737598,-73.978604


In [186]:
# Read the file containing the overlapping rectangles for Manhattan
wrapper = io.TextIOWrapper(project.get_file("BrooklynLatLong.csv"), encoding='utf-8')
dfBrooklynLatLon = pd.read_csv(wrapper)
dfBrooklynLatLon.head()

,SWLAT,SWLON,NELAT,NELON
0,40.586324,-74.049198,40.637928,-73.894187
1,40.636756,-74.048168,40.678819,-73.871871
2,40.679470,-74.033233,40.703939,-73.886806
3,40.700035,-74.016925,40.729571,-73.898994


In [187]:
# Make an enpty dataframe that will contains all the venues found
# for all the categories of interest, in all the rectangles in Manhattan
dfAllVenues = pd.DataFrame()

# Merge the two Lat/Lon CSV-based DataFrames
dfTwoBoroughsLatLon = pd.concat([dfManhattanLatLon, dfBrooklynLatLon])

for aCategory in categoriesToFind:
    # Get the ID of the category
    categoryName = aCategory
    categoryID = dfAllCategories.loc[categoryName]['id']
    print('Retrieving ' + categoryName)

    # Iterate over the rectangles, retrieving a list of venues for the category
    for row in dfTwoBoroughsLatLon.itertuples():
        swLat = row.SWLAT
        swLon = row.SWLON
        neLat = row.NELAT
        neLon = row.NELON

        url = 'https://api.foursquare.com/v2/venues/search?intent=browse&client_id={}&client_secret={}&sw={},{}&ne={},{}&v={}&categoryId={}&'\
    .format(CLIENT_ID, CLIENT_SECRET, swLat, swLon, neLat, neLon, VERSION, categoryID)
        results = requests.get(url).json()

        # assign relevant part of JSON to venues
        venues = results['response']['venues']

        # tranform venues into a dataframe
        dfOneRect = json_normalize(venues)
        dfOneRect['category'] = categoryName
#        print('Retrieved ' + str(dfOneRect.shape[0]) + ' for categoryID=' + categoryID)
#        print()

        # append that dataframe to the one containing all rectangles' venues
        dfAllVenues = pd.concat([dfAllVenues, dfOneRect], sort=False)

dfAllVenues.drop(dfAllVenues.columns.difference(['category', 'name', 'location.postalCode', \
                                'location.address', 'location.city', 'location.lat', 'location.lng']), 1, inplace=True)

newColumnNames = ['address', 'city', 'lat', 'lon', 'zip', 'name',  'category']
dfAllVenues.columns = newColumnNames
dfAllVenues.columns
## Drop rows where the venue doesn't have a zip code, because we need that for further processing
#dfAllVenues.dropna(subset=['location.postalCode'], inplace=True)
#dfAllVenues.shape
print(dfAllVenues.shape)
dfAllVenues.head()

Retrieving Movie Theater
Retrieving Juice Bar
(789, 7)


,address,city,lat,lon,zip,name,category
0,11 Fulton St,New York,40.706611,-74.003184,10038,iPic Theaters at Fulton Market,Movie Theater
1,63 Wall St,New York,40.706031,-74.008814,10005,Royal Theater Cinema,Movie Theater
2,NaN,NaN,40.706855,-74.010915,10005,Unknown,Movie Theater
3,55 Broad St,New York,40.705091,-74.011511,10004,Thundergulch,Movie Theater
4,20 West St,New York,40.706889,-74.015796,10004,The Downtown Athletic Club Screening Room,Movie Theater


In [188]:
# Some results from FourSquare have lat/lon, but no zip code
# Use reverse geocoding from Bing to retrieve the missing zip codes,
# and update dfAllVenues

dfNoZip = dfAllVenues[dfAllVenues['zip'].isna()]

print("Before processing, there are " + str(len(dfNoZip)) + " rows in dfAllVenues without zip codes")

# Get the column index of 'zip' so we can use the faster "iat" method
zipIndex = dfAllVenues.columns.get_loc("zip")

for index, row in dfNoZip.iterrows():    
    try:
        lat = row.lat
        lon = row.lon
        bingURL = 'https://dev.virtualearth.net/REST/v1/locationrecog/{},{}?key={}&r=.5&distanceUnit=mi&includeEntityTypes=address&output=json'\
        .format(lat,lon,bingMapsKey)

        results = requests.get(bingURL).json()
        theZip = results['resourceSets'][0]['resources'][0]['addressOfLocation'][0]['postalCode']
        dfAllVenues.at[index, 'zip'] = theZip

    #    print('Zip is {} for {}, {}'.format(theZip, lat, lon))
    except:
        pass # doing nothing on exception, we'll filter out rows w/o zip codes later
    
# Now repeat for empty string zip codes
dfNoZip = dfAllVenues[dfAllVenues['zip'].str.len().eq(0)]
for index, row in dfNoZip.iterrows():    
    try:
        lat = row.lat
        lon = row.lon
        bingURL = 'https://dev.virtualearth.net/REST/v1/locationrecog/{},{}?key={}&r=.5&distanceUnit=mi&includeEntityTypes=address&output=json'\
        .format(lat,lon,bingMapsKey)

        results = requests.get(bingURL).json()
        theZip = results['resourceSets'][0]['resources'][0]['addressOfLocation'][0]['postalCode']
        dfAllVenues.at[index, 'zip'] = theZip

        #print('Zip is {} for {}, {}'.format(theZip, lat, lon))
    except:
#        print(results)
        pass # doing nothing on exception, we'll filter out rows w/o zip codes later

dfNoZip = dfAllVenues[dfAllVenues['zip'].isna()]
dfNoZip2 = dfAllVenues[dfAllVenues['zip'].str.len().eq(0)]
print("After processing, there are " + str(len(dfNoZip) + len(dfNoZip2)) + " rows in dfAllVenues without zip codes")

# Ensure that zip code is a string
dfAllVenues.zip = dfAllVenues.zip.astype(str)


Before processing, there are 105 rows in dfAllVenues without zip codes
After processing, there are 0 rows in dfAllVenues without zip codes


### Read in Zip Codes

In [189]:
# Read in the CSV file 'Brooklyn and Manhattan Zip Codes By Neighborhood.csv'.
# I created this file by combining data from several files. The CSV file
# has these columns:
#  Borough: Either Brooklyn or Manhattan
#  SubBoroNumber: An identifier created by New York City to identify sections of each borough
#  Name: The name of the neighborhood(s) (sub-boro)
#  ZipCode: One of the zip codes in the neighborhood(s)
# There are multiples lines for most neighborhoods in the CSV file, because
# most neighborhoods contain more than one zip code

import io

wrapper = io.TextIOWrapper(project.get_file("Brooklyn and Manhattan Zip Codes By Neighborhood.csv"), encoding='utf-8')
dfNeighborhoodZipcodes = pd.read_csv(wrapper)

# Lets convert ZipCode from int to string, so that zips with leading zeroes (e.g., 02123) work.
# NYC doesn't have any zips with leading zeroes, but it's good data management
# to ensure that code will work with any valid data that's out there, not just the 
# particular data you're using at the moment
dfNeighborhoodZipcodes.ZipCode = dfNeighborhoodZipcodes.ZipCode.astype(str)

# Ensure that Borough and Name columns are strings
dfNeighborhoodZipcodes.Borough = dfNeighborhoodZipcodes.Borough.astype(str)
dfNeighborhoodZipcodes.Name = dfNeighborhoodZipcodes.Name.astype(str)

# Let's test our int-to-string conversion by checking which neighborhood the 
# zip code "11201" (a string, not the int 11201) is in
print(dfNeighborhoodZipcodes.loc[dfNeighborhoodZipcodes['ZipCode'] == "11201"])

    Borough  SubBoroNumber                           Name ZipCode
2  Brooklyn  2              Brooklyn Heights, Fort Greene  11201 


In [190]:
# Read in a CSV file that has the population of each neighborhood
# The CSV file has these columns:
#  Borough: Either Brooklyn or Manhattan
#  SubBoroNumber: An identifier created by New York City to identify sections of each borough
#  Name: The name of the neighborhood(s) (sub-boro)
#  Population: The number of people living in the neighborhood as of the 2010 census
# We can match rows in this data to rows in the zip code file by matching Borough andSubBohoNumber,
# or more directly by matching the neighborhood name

wrapper = io.TextIOWrapper(project.get_file("Brooklyn and Manhattan Population By Neighborhood.csv"), encoding='utf-8')
dfNeighborhoodPopulation = pd.read_csv(wrapper)
dfNeighborhoodPopulation.head()

,Borough,SubBoroNumber,Name,Population
0,Brooklyn,1,"Williamsburg, Greenpoint",173083
1,Brooklyn,2,"Brooklyn Heights, Fort Greene",99617
2,Brooklyn,3,Bedford Stuyvesant,152985
3,Brooklyn,4,Bushwick,112634
4,Brooklyn,5,"East New York, Starrett City",182896


In [191]:
dfAllVenues.head()

,address,city,lat,lon,zip,name,category
0,11 Fulton St,New York,40.706611,-74.003184,10032,iPic Theaters at Fulton Market,Movie Theater
1,63 Wall St,New York,40.706031,-74.008814,11210,Royal Theater Cinema,Movie Theater
2,NaN,NaN,40.706855,-74.010915,10025,Unknown,Movie Theater
3,55 Broad St,New York,40.705091,-74.011511,10034,Thundergulch,Movie Theater
4,20 West St,New York,40.706889,-74.015796,10034,The Downtown Athletic Club Screening Room,Movie Theater


In [192]:
# Now that we have the venues (with zip codes), and a way to look up
# a neighborhood for a particular zip code, iterate over dfAllVenues,
# adding the neighborhood to each entry.

# Start by adding the columns that exist in dfNeighborhoodZipcodes to dfAllVenues
dfAllVenues["Borough"] = ""
dfAllVenues["SubBoroNumber"] = 0
dfAllVenues["Neighborhood"] = ""

for index, row in dfAllVenues.iterrows():    
    zipCode = row.zip
    name = row.name
    
    # Get the neighborhood this zip code corresponds to
    dfNeighborhood = dfNeighborhoodZipcodes.loc[dfNeighborhoodZipcodes['ZipCode'] == zipCode]
    if (len(dfNeighborhood) == 0):
#        print("No neighborhood for zip code " + zipCode + " for venue " + name)
        print("No neighborhood for zip code '" + zipCode + "'")
        print(row)
    else:
        borough = dfNeighborhood.iloc[0]["Borough"]
        subBoro = dfNeighborhood.iloc[0]["SubBoroNumber"]
        neighborhood = dfNeighborhood.iloc[0]["Name"]

        dfAllVenues.at[index, 'Borough'] = borough
        dfAllVenues.at[index, 'SubBoroNumber'] = subBoro
        dfAllVenues.at[index, 'Neighborhood'] = neighborhood

dfAllVenues.head()

,address,city,lat,lon,zip,name,category,Borough,SubBoroNumber,Neighborhood
0,11 Fulton St,New York,40.706611,-74.003184,10032,iPic Theaters at Fulton Market,Movie Theater,Manhattan,12,"Washington Heights, Inwood"
1,63 Wall St,New York,40.706031,-74.008814,11210,Royal Theater Cinema,Movie Theater,Brooklyn,17,"East Flatbush, Rugby, Farragut"
2,NaN,NaN,40.706855,-74.010915,10025,Unknown,Movie Theater,Manhattan,7,"West Side, Upper West Side"
3,55 Broad St,New York,40.705091,-74.011511,10034,Thundergulch,Movie Theater,Manhattan,12,"Washington Heights, Inwood"
4,20 West St,New York,40.706889,-74.015796,10034,The Downtown Athletic Club Screening Room,Movie Theater,Manhattan,12,"Washington Heights, Inwood"


In [193]:
# Clean up the data, removing any rows in dfAllVenues that have SubBoroNumber == 0, because
# that means their zip code did not match up with any of the zip codes in the neighborhoods
# we're including.
numRowsBefore = len(dfAllVenues)

# Get indexes for which column SubBoroNumber is zero
indexNames = dfAllVenues[dfAllVenues['SubBoroNumber'] == 0].index
 
# Delete these row indexes from dataFrame
dfAllVenues.drop(indexNames , inplace=True)
numRowsAfter = len(dfAllVenues)
numRowsRemoved = numRowsBefore - numRowsAfter
print("Removed " + str(numRowsRemoved) + " rows that are in neighborhoods we're not including")


Removed 0 rows that are in neighborhoods we're not including


In [194]:
# Create a new dataframe that has the borough, sub-boro, and the groupby count
# of venues in that borough/sub-boro
dfVenueCount = dfAllVenues.groupby(['Borough', 'SubBoroNumber']).name.agg('count').to_frame('VenueCount').reset_index()

In [195]:
dfVenueCount

,Borough,SubBoroNumber,VenueCount
0,Brooklyn,3,13
1,Brooklyn,7,50
2,Brooklyn,10,10
3,Brooklyn,15,15
4,Brooklyn,17,112
5,Manhattan,1,120
6,Manhattan,2,58
7,Manhattan,4,10
8,Manhattan,6,21
9,Manhattan,7,106


In [196]:
# Merge the two dataframes: dfNeighborhoodPopulation and dfVenueCount
# They both have a Borough and a SubBoroNumber column, so we'll merge
# on that combination. We'll do a left merge, which will include all
# the columns from dfNeighborhoodPopulation, and add the VenueCount
# column
dfNeighborhoodPopulation = pd.merge(dfNeighborhoodPopulation, dfVenueCount, how='left', on =['Borough', 'SubBoroNumber'])

#dfNeighborhoodPopulation.VenueCount = dfNeighborhoodPopulation.VenueCount.astype(int)

dfNeighborhoodPopulation.head()

,Borough,SubBoroNumber,Name,Population,VenueCount
0,Brooklyn,1,"Williamsburg, Greenpoint",173083,NaN
1,Brooklyn,2,"Brooklyn Heights, Fort Greene",99617,NaN
2,Brooklyn,3,Bedford Stuyvesant,152985,13.0
3,Brooklyn,4,Bushwick,112634,NaN
4,Brooklyn,5,"East New York, Starrett City",182896,NaN


In [197]:
dfPop = dfNeighborhoodPopulation

# A neighborhood that doesn't have any venues will have NaN for VenueCount.
# Convert those to zero
values = {'VenueCount': 0}
dfPop.fillna(value=values, inplace=True)

# Calculate the number of venues per 10,000 residents of each neighborhood
dfPop["VenueDensity"] = dfNeighborhoodPopulation["VenueCount"] / (dfNeighborhoodPopulation["Population"] / 10000)

# Print the neighborhoods with the highest venue densities
dfPop = dfPop.sort_values('VenueDensity', ascending=False)

dfPop.head(10)

,Borough,SubBoroNumber,Name,Population,VenueCount,VenueDensity
18,Manhattan,1,"Battery Park City, Tribeca",60978,120.0,19.679229
29,Manhattan,12,"Washington Heights, Inwood",190020,145.0,7.630776
16,Brooklyn,17,"East Flatbush, Rugby, Farragut",155252,112.0,7.214078
19,Manhattan,2,"Greenwich Village, Soho",90016,58.0,6.443299
24,Manhattan,7,"West Side, Upper West Side",209084,106.0,5.069733
25,Manhattan,8,Upper East Side,219920,89.0,4.046926
6,Brooklyn,7,"Sunset Park, Windsor Terrace",126230,50.0,3.961024
26,Manhattan,9,"Manhattanville, Hamilton Heights",110193,40.0,3.629995
23,Manhattan,6,"Stuyvesant Town, Turtle Bay",142745,21.0,1.471155
21,Manhattan,4,"Chelsea, Clinton",103245,10.0,0.968570


## Geopandas

In [396]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 20)

In [397]:
import geopandas as gpd

# Read the zip code/boundary file into memory
nyBoundaryFile = project.get_file('NYCZipcodeBoundaries.json')

# Turn the zip code/boundary file into a geopandas dataframe
nyBoundaries = gpd.read_file(nyBoundaryFile)
nyBoundaries.head()

# select the columns that you with to use for the dissolve and that will be retained
nyBoundaries2 = nyBoundaries[['postalCode', 'borough', 'PO_NAME', 'geometry']]
#print(nyBoundaries2['PO_NAME'] + "  " + nyBoundaries2['postalCode'])


In [398]:
# Drop the rows from the geojson file that are for zip code we aren't using
nyBoundaries2 = nyBoundaries2[nyBoundaries2.postalCode.isin(dfNeighborhoodZipcodes['ZipCode'])]
nyBoundaries2.head()

,postalCode,borough,PO_NAME,geometry
21,11221,Brooklyn,Brooklyn,"POLYGON ((-73.93822325191614 40.68389331961758, -73.9438911382472 40.68323950227984, -73.94564165707669 40.69203346946512, -73.93413376737952 40.69335436373854, -73.93479698827541 40.69673661671486, -73.9352226413318 40.69698656384033, -73.93226765371799 40.69731469442813, -73.93324668417745 40.69785561272368, -73.93076911862683 40.69891104645372, -73.9305202888139 40.69877230313472, -73.92758773780713 40.70176259998695, -73.92308168149715 40.69920069630066, -73.92163115701487 40.70067558024493, -73.92087956801677 40.70024736678035, -73.92232800953694 40.6987721198053, -73.91848721378724 40.69658602292844, -73.91781911962863 40.69620772862452, -73.91636718052304 40.69768090418263, -73.90885282871758 40.69340445130968, -73.91604598744264 40.68607208871235, -73.91804606753695 40.687213268776, -73.9178493642506 40.68623318739356, -73.93822325191614 40.68389331961758))"
25,11216,Brooklyn,Brooklyn,"POLYGON ((-73.94564165707669 40.69203346946512, -73.9438911382472 40.68323950227984, -73.93822325191614 40.68389331961758, -73.93739763108535 40.67972992244106, -73.94120864001327 40.67993836946285, -73.94160931928725 40.67564320442343, -73.94714602322288 40.67594438109577, -73.94773123647958 40.6697047078836, -73.9550528799369 40.67010365000868, -73.95797316022487 40.6706572921663, -73.95516306696332 40.67852074712599, -73.95713186042737 40.68846315202325, -73.95131871695537 40.68913231750482, -73.95175746790927 40.69132933067435, -73.94564165707669 40.69203346946512))"
27,11233,Brooklyn,Brooklyn,"POLYGON ((-73.93822325191614 40.68389331961758, -73.9178493642506 40.68623318739356, -73.91804606753695 40.687213268776, -73.9034668872538 40.67889653817026, -73.90329155804092 40.67786792438706, -73.90346256333866 40.6760158063333, -73.90240783422718 40.67587541049135, -73.90438871964855 40.67517137038907, -73.925653595542 40.66582990624871, -73.92580501111077 40.666456157208, -73.92499590519674 40.67473589594239, -73.92776143466391 40.67488742035348, -73.92768791018021 40.67566288431892, -73.93045708472673 40.67581876680987, -73.93020074773672 40.67804022252579, -73.92905727071383 40.67927645406513, -73.93739763108535 40.67972992244106, -73.93822325191614 40.68389331961758))"
29,11213,Brooklyn,Brooklyn,"POLYGON ((-73.93739763108535 40.67972992244106, -73.92905727071383 40.67927645406513, -73.93020074773672 40.67804022252579, -73.93045708472673 40.67581876680987, -73.92768791018021 40.67566288431892, -73.92776143466391 40.67488742035348, -73.92499590519674 40.67473589594239, -73.92580501111077 40.666456157208, -73.925653595542 40.66582990624871, -73.93065206871213 40.66363834202987, -73.93726377174885 40.66319183016009, -73.93715846706962 40.66375261786932, -73.94269628450726 40.66405515298856, -73.94252842813368 40.66583793996203, -73.94806707556559 40.66613891733024, -73.94714602322288 40.67594438109577, -73.94160931928725 40.67564320442343, -73.94120864001327 40.67993836946285, -73.93739763108535 40.67972992244106))"
30,11212,Brooklyn,Brooklyn,"POLYGON ((-73.9029413249695 40.67084010909053, -73.90222652406146 40.66833055629683, -73.9019226438036 40.66837507399539, -73.90072246594312 40.6637786795063, -73.90091625191538 40.66373345964472, -73.89946353296122 40.6586827358688, -73.89982259690147 40.65729128796997, -73.90044831154252 40.6572521076026, -73.90127968809493 40.65643865647322, -73.90317307067548 40.65530153548355, -73.91088756724415 40.6541599129303, -73.92172993981653 40.65438234063733, -73.93153753823071 40.66323653070145, -73.90347422563424 40.6755067025594, -73.90325835602846 40.6733963820095, -73.90361711807671 40.67334411191762, -73.9029413249695 40.67084010909053))"


In [399]:
# Now merge the neighborhood column from dfNeighborhoodZipcodes into nyBoundaries2, matching on zip code

# First, let's change 'postalCode' to 'ZipCode' since that's what dfNeighborhoodZipcodes calls it
newColumnNames = list(nyBoundaries2.columns)
for idx, item in enumerate(newColumnNames):
   if 'postalCode' in item:
       newColumnNames[idx] = 'ZipCode'
nyBoundaries2.columns = newColumnNames

nyBoundaries2 = pd.merge(nyBoundaries2, dfNeighborhoodZipcodes, how='inner', on =['ZipCode'])

# dissolve the state boundary by region 
nyBoundaries3 = nyBoundaries2.dissolve(by='Name')

# Turn the Name column from an index back into a regular column we can access
nyBoundaries3.reset_index(inplace=True)

print("Merged the geographic boundaries of " + str(len(nyBoundaries2)) + " zip codes into " + str(len(nyBoundaries3)) + " neighborhoods")

# We have to add the neighborhood name back into nyBoundaries3. It was
# lost during the dissolve
#nyBoundaries3['Neighborhood'] = dfNeighborhoodZipcodes.loc[dfNeighborhoodZipcodes['ZipCode'] == nyBoundaries3.iloc[0]['ZipCode']].Name


Merged the geographic boundaries of 97 zip codes into 30 neighborhoods


## Maps

In [400]:
# Get the lat/lon of the center of Manhattan & Brooklyn
# I visually determind the 'center' of Manhattan to be about where the Guggenheim Museum is,
# just east of Central Park, so I query Bing to get its lat/lon
bingURL = 'http://dev.virtualearth.net/REST/v1/Locations/{}?maxResults=1&key={}'.format("Guggenheim Museum, NY", bingMapsKey)
results = requests.get(bingURL).json()
manhattanCenter = results['resourceSets'][0]['resources'][0]['geocodePoints'][0]['coordinates']
print("Manhattan center: " + str(manhattanCenter))

# Various online sources pinpoint the center of Brookly as Brooklyn College, which is at
# 2900 Bedford Avenue.
bingURL = 'http://dev.virtualearth.net/REST/v1/Locations/{}?maxResults=1&key={}'.format("2900 Bedford Avenue, Brookly, NY", bingMapsKey)
results = requests.get(bingURL).json()
brooklynCenter = results['resourceSets'][0]['resources'][0]['geocodePoints'][0]['coordinates']
print("Brookly center: " + str(brooklynCenter))


Manhattan center: [40.78292465209961, -73.95895385742188]
Brookly center: [40.631289, -73.952159]


In [401]:
def neighborhoods_style(feature):
    return { 'color': 'blue', 'fill': False }

In [402]:
import folium
from folium import plugins
from folium.plugins import HeatMap

map_manhattan = folium.Map(location=manhattanCenter, zoom_start=13, tiles='Mapbox Bright')
folium.TileLayer('cartodbpositron').add_to(map_manhattan)
#HeatMap(restaurant_latlons).add_to(map_berlin)
#folium.Marker(manhattanCenter).add_to(map_manhattan)
#folium.Circle(manhattanCenter, radius=1000, fill=False, color='white').add_to(map_berlin)
#folium.Circle(manhattanCenter, radius=2000, fill=False, color='white').add_to(map_berlin)
#folium.Circle(manhattanCenter, radius=3000, fill=False, color='white').add_to(map_berlin)
gjson = nyBoundaries3.to_crs(epsg='4326').to_json()
folium.GeoJson(gjson, style_function=neighborhoods_style, name='geojson').add_to(map_manhattan)
#map_manhattan

In [403]:
dfAllVenues.head()

,address,city,lat,lon,zip,name,category,Borough,SubBoroNumber,Neighborhood
0,11 Fulton St,New York,40.706611,-74.003184,10032,iPic Theaters at Fulton Market,Movie Theater,Manhattan,12,"Washington Heights, Inwood"
1,63 Wall St,New York,40.706031,-74.008814,11210,Royal Theater Cinema,Movie Theater,Brooklyn,17,"East Flatbush, Rugby, Farragut"
2,NaN,NaN,40.706855,-74.010915,10025,Unknown,Movie Theater,Manhattan,7,"West Side, Upper West Side"
3,55 Broad St,New York,40.705091,-74.011511,10034,Thundergulch,Movie Theater,Manhattan,12,"Washington Heights, Inwood"
4,20 West St,New York,40.706889,-74.015796,10034,The Downtown Athletic Club Screening Room,Movie Theater,Manhattan,12,"Washington Heights, Inwood"


In [404]:
# Create a heat map with a marker for each venue, with the
# neighborhood boundaries overlayed on the map

# Make sure lat/lon are floats
dfAllVenues['lat'] = dfAllVenues['lat'].astype(float)
dfAllVenues['lon'] = dfAllVenues['lon'].astype(float)

# Make a list of lat/lon points of all the venues
heatMapData = [[row['lat'],row['lon']] for index, row in dfAllVenues.iterrows()]
HeatMap(heatMapData).add_to(map_manhattan)

   
# Add the centroid of each neighborhood to its geopandas DataFrame so we know
# where to put a text label identifying the neighborhood
nyBoundaries3["neighborhoodCentroid"] = nyBoundaries3.centroid
for index, row in nyBoundaries3.iterrows():
    textMarker = folium.Marker(location=[row.neighborhoodCentroid.y, row.neighborhoodCentroid.x], popup=row.Name)
    map_manhattan.add_child(textMarker)

map_manhattan

## Clustering

In [405]:
dfAllVenues.columns

Index(['address', 'city', 'lat', 'lon', 'zip', 'name', 'category', 'Borough',
       'SubBoroNumber', 'Neighborhood'],
      dtype='object')

In [406]:
venues_onehot = pd.get_dummies(dfAllVenues[['category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
venues_onehot['lat'] = dfAllVenues['lat'] 
venues_onehot['lon'] = dfAllVenues['lon'] 

venues_onehot.head()

,Juice Bar,Movie Theater,lat,lon
0,0,1,40.706611,-74.003184
1,0,1,40.706031,-74.008814
2,0,1,40.706855,-74.010915
3,0,1,40.705091,-74.011511
4,0,1,40.706889,-74.015796


In [407]:
from sklearn.cluster import KMeans 

# set number of clusters
kclusters = 10

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_onehot)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

In [408]:
# add clustering labels to the dataframe, and add back the venue names
venues_onehot.insert(0, 'ClusterLabel', kmeans.labels_)
venues_onehot['name'] = dfAllVenues['name'] 
venues_onehot['category'] = dfAllVenues['category'] 
venues_onehot.head()


,ClusterLabel,Juice Bar,Movie Theater,lat,lon,name,category
0,1,0,1,40.706611,-74.003184,iPic Theaters at Fulton Market,Movie Theater
1,1,0,1,40.706031,-74.008814,Royal Theater Cinema,Movie Theater
2,1,0,1,40.706855,-74.010915,Unknown,Movie Theater
3,1,0,1,40.705091,-74.011511,Thundergulch,Movie Theater
4,1,0,1,40.706889,-74.015796,The Downtown Athletic Club Screening Room,Movie Theater


In [409]:
pd.value_counts(venues_onehot['ClusterLabel'])

1    158
3    156
0    127
2    99 
4    71 
8    48 
5    43 
6    37 
9    30 
7    20 
Name: ClusterLabel, dtype: int64

In [410]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

from folium.plugins import MarkerCluster

# create map
#map_clusters = folium.Map(location=manhattanCenter, zoom_start=11)
map_clusters = map_manhattan

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#map_clusters

In [411]:
# Create a list of N MarkerCluster objects
# We'll treat this list like an array, using it to add
# venues to the appropriate marker cluster for the cluster map
numClusters = len(venues_onehot['ClusterLabel'].unique())
mcList = [MarkerCluster] * numClusters
nIndex = 0
for mc in mcList:
    mcList[nIndex] = MarkerCluster()
    nIndex = nIndex + 1


In [412]:
# add cluster markers to the map, with a cluster child for each top venue
# iterate over the rows in the DataFrame
for clusterRow in venues_onehot.itertuples():
    lat = clusterRow.lat
    lon = clusterRow.lon
    name = clusterRow.name
    category = clusterRow.category
    clusterNumber = clusterRow.ClusterLabel
    mc = mcList[clusterNumber]

    markerColor = dfMapMarkers.loc[category].color
    markerIconPrefix = dfMapMarkers.loc[category].icon_prefix
    markerIconName = dfMapMarkers.loc[category].icon_name
    thePopup = folium.Popup(name)
#    mc.add_child(folium.Marker(location=[lat,  lon], popup=thePopup, icon=folium.Icon(color='red', icon='info-sign')))
#    mc.add_child(folium.Marker(location=[lat,  lon], popup=thePopup, icon=folium.Icon(color='red', icon='glyphicon-music')))
    mc.add_child(folium.Marker(location=[lat,  lon], popup=thePopup, icon=folium.Icon(color=markerColor, prefix=markerIconPrefix, icon=markerIconName)))

for mc in mcList:
    map_clusters.add_child(mc)
    
map_clusters